### 1. Char-RNN을 사용해 셰익스피어 문체 생성하기

#### 1) 훈련 데이터셋 만들기

In [8]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

In [ ]:
# 데이터 불러오기

shakespeare_url = 'https://homl.info/shakespeare'
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)

with open(filepath) as f:
    shakespeare_text = f.read()

1122304/1115394 [==============================] - 0s 0us/step


In [ ]:
shakespeare_text[:50]

'First Citizen:\nBefore we proceed any further, hear'

In [ ]:
len(shakespeare_text)

1115394

In [ ]:
# 글자를 정수(ID)로 인코딩하기 (ID는 1부터 시작)

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) # 글자수준으로
tokenizer.fit_on_texts(shakespeare_text)  # 이 텍스트에 훈련하기

In [ ]:
# tokenizer 작동 확인

tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [ ]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [ ]:
max_id = len(tokenizer.word_index) # 고유 글자 개수
dataset_size = tokenizer.document_count # 전체 글자 개수

max_id, dataset_size

(39, 1115394)

In [ ]:
# 전체 텍스트를 인코딩하여 글자를 ID로 나타내기
# ID를 0부터 시작하기 위해 1을 빼줌

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

# [encoded]를 해주는 이유 : 그냥 encoded하면 [[19, ...]]이렇게 되는데,
# 리스트 하나를 빼주려고

In [ ]:
encoded

array([19,  5,  8, ..., 20, 26, 10])

#### 2) 순차 데이터셋 나누기

In [ ]:
# 90%를 훈련 데이터셋으로 사용하기

train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
  # 한 번에 한 글자씩 반환하는 tf.data.Dataset 객체 

In [ ]:
# tf.data.Dataset.from_tensor_slices 작동 방식

# t = tf.constant([[1, 2], [3, 4]])
# ds = tf.data.Dataset.from_tensors(t)   # [[1, 2], [3, 4]]

# t = tf.constant([[1, 2], [3, 4]])
# ds = tf.data.Dataset.from_tensor_slices(t)   # [1, 2], [3, 4]

#### 3) 순차 데이터를 윈도 여러개로 자르기
<img src='img/16_1.png' width='400'>

In [ ]:
### 윈도로 끊어주기

n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

## drop_remainder=False 이면
# ds = tf.data.Dataset.range(6) 
# ds = ds.window(5, shift=1, drop_remainder=False)
# for d in ds:
#     print(list(d.as_numpy_iterator()))

# [[0, 1, 2, 3, 4],
# [1, 2, 3, 4, 5],
# [2, 3, 4, 5],
# [3, 4, 5],
# [4, 5],
# [5]]

In [ ]:
### 중첩 데이터(nested data)를 flat 데이터로 만들어주기

# 이때, batch() 함수를 사용해 윈도 길이로 끊어주기
# {{1, 2}, {3, 4, 5, 6}} (중첩 data) -> {[1, 2], [3, 4], [5, 6]} (flat data)

dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [ ]:
### 훈련 data 섞고, 배치로 만든 후, X와 y 분리하기
# 섞는 이유 : data가 iid일 때, 경사하강법이 가장 잘 작동하기 때문

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
    # shuffle의 파라미터는 buffer_size 적당히 큰 수를 해줘야 랜덤하게 잘 뽑음
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [ ]:
### 원핫 인코딩

dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

#### 4) 모델만들고 훈련하기

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                    dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                    dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [ ]:
# 오래 걸림

history = model.fit(dataset, epochs=20) 

Epoch 1/20
  21905/Unknown - 12710s 580ms/step - loss: 1.6434

#### 5) 모델 사용하기

In [ ]:
def preprocess(text):
    X = np.array(tokenizer.text_to_sequences(text)) - 1
    return tf.one_hot(X, max_id)

# 토큰화 해주고, 원핫인코딩 해주기

In [ ]:
# 에측하기

X_new = preprocess(['How are yo'])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_text(Y_pred + 1)[0][-1]  # 첫번째 문장, 마지막 글자

#### 6) 가짜 셰익스피어 텍스트 생성하기

In [ ]:
# 다음 글자 예측하는 함수

def next_chat(text, temperature=1):
        # temperature이 0에 가까울수록 높은 확률을 가진 글자를 선택 
        # (확률분포를 더 두드러지게, 1에 가까우면 그냥 원래의 확률분포로)
        # (원래 predict는 0~1의 출력. 여기에 로그를 취하고 0에 가까운 temperature로 
        # 나눈 후 다시 지수 함수로 복원하면 작았던 확률이 더 크게 작아짐)

    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescale_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
        # 추정한 확률(로짓)을 기반으로 랜덤하게 클래스를 샘플링
        # 이렇게 안하면 같은 단어가 계속 반복되는 경우가 많음
    return tokenizer.sequences._to_text(char_id.numpy())[0]

In [ ]:
# next_char을 반복 호출하여 텍스트에 추가해나가는 함수

def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_chart(text, temperatrue)
    return text

In [ ]:
print(complete_text('t', temperature=0.2), '\n')
print(complete_text('w', temperature=1), '\n')
print(complete_text('w', temperature=2), '\n')

# 여기서는 1에 가까운 온동에서 잘 작동함
# 더 좋은 텍스트를 생성하려면 GRU층과 층의 뉴런 수를 늘리고 더 오래 훈련하거나 규제를 추가

#### 7) 상태가 있는 RNN (stateful RNN)
- 입력 데이터가 순차적이고 겹치기 않아야!  
  ex) 첫 번째 배치는 윈도 1 ~ 32, 두 번째 배치는 윈도 33 ~ 64  
  <img src='img/16_2.png' width='400'>

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
# 배치만드려면 아래와 같은 코드 (데이터를 32개로 나누기)

# batch_size = 32
# encoded_parts = np.array_split(encoded[:train_size], batch_size)

# datasets = []
# for encoded_part in encoded_parts:
#     dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
#     dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
#     dataset = dataset.flat_map(lambda window: window.batch(window_length))
#     datasets.append(dataset)
# dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
# dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
# dataset = dataset.map(
#     lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
# dataset = dataset.prefetch(1)

In [ ]:
# 모델 구성 및 학습

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[1, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation='softmax'))
])

In [ ]:
# 모델 재설정해주는 콜백함수

class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [ ]:
steps_per_epoch = train_size // batch_size // n_steps

In [ ]:
history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50,
                    callbacks=[ResetStatesCallback()])

Epoch 1/50
313/313 [==============================] - 75s 229ms/step - loss: 3.0333
Epoch 2/50
313/313 [==============================] - 71s 226ms/step - loss: 2.4841
Epoch 3/50
313/313 [==============================] - 71s 228ms/step - loss: 2.2787
Epoch 4/50
313/313 [==============================] - 74s 236ms/step - loss: 2.2314
Epoch 5/50
313/313 [==============================] - 70s 223ms/step - loss: 2.1705
Epoch 6/50
313/313 [==============================] - 69s 219ms/step - loss: 2.1746
Epoch 7/50
313/313 [==============================] - 70s 222ms/step - loss: 2.0817
Epoch 8/50
313/313 [==============================] - 69s 222ms/step - loss: 2.0439
Epoch 9/50
313/313 [==============================] - 70s 224ms/step - loss: 2.0255
Epoch 10/50
313/313 [==============================] - 70s 222ms/step - loss: 1.8959
Epoch 11/50
313/313 [==============================] - 69s 221ms/step - loss: 2.0144
Epoch 12/50
313/313 [==============================] - 69s 219ms/step - lo

### 감성 분석
- IMDb 리뷰 데이터 (영화 리뷰 50,000개 & 이진 분류 - 0(부정)/1(긍정))

In [ ]:
# 데이터 적재

(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()
X_train[0][:10]  # 전처리 되서 넘파이 정수 배열로 표현됨
                 # 구두점을 모두 제거하고 단어는 소문자로 변환한 다음, 공백으로 나누어
                 # 빈도에 따라 인덱스를 붙임 (낮은 정수가 자주 등장하는 단어)
                 # 단, 0:패딩토큰, 1:SOS토큰, 2:알수없는 단어

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [ ]:
# 리뷰 내용을 보기위해 디코딩해보기

word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}

# word_index에는 단어들로만 이루어져 있는데(1:'the'), 실제 인코딩은 1:<sos>임
# 디코딩해주기 위해서 0, 1, 2에 해당하는 토큰들도 넣어주기
for id_, token in enumerate(('<pad>', '<sos>', '<unk>')):
    id_to_word[id_] = token

' '.join([id_to_word[id_] for id_ in X_train[0][:10]])

1646592/1641221 [==============================] - 0s 0us/step


'<sos> this film was just brilliant casting location scenery story'

In [9]:
# 전처리를 모델 자체에 포함시키기

import tensorflow_datasets as tfds

In [2]:
datasets, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)
    # as_supervised=True하면 데이터가 (input, label)형태로 나옴
train_size = info.splits['train'].num_examples

In [3]:
### 전처리 함수

def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)  # 리뷰에서 처음 300글자만 남김 (속도 up)
    X_batch = tf.strings.regex_replace(X_batch, b'<br\\s*/?>', b' ') # <br /> 공백으로
    X_batch = tf.strings.regex_replace(X_batch, b'[^a-zA-Z]', b' ')  # 문자, 작은따옴표아니면 공백으로
    X_batch = tf.strings.split(X_batch)  # 띄어쓰기 기준으로 split (래그드 텐서가 반환됨)
    return X_batch.to_tensor(default_value=b'<pad'), y_batch  # 밀집 텐서(일반 텐서)로 바꾸고 
                                # 동일한 길이의 텐서가 되도록 <pad> 토큰으로 패딩하기
                                # default_value를 지정하지 않으면 빈 바이트 문자열로 패딩

In [4]:
### 어휘사전 구축하기

from collections import Counter
vocabulary = Counter()

In [10]:
for X_batch, y_batch in datasets['train'].batch(32).map(preprocess):
                                        # batch 사이즈가 32
    for review in X_batch:
        vocabulary.update(list(review.numpy())) # 배치마다 리스트로 업데이트

In [11]:
vocabulary.most_common()[:3]  # 가장 많이 등장하는 단어 한 번 확인해보기

[(b'<pad', 224494), (b'the', 61156), (b'a', 38569)]

In [12]:
### 어휘 사전에 가장 많이 등장하는 단어 10,000개만 남기고 삭제하기

vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

In [13]:
### oov 버킷을 사용하는 룩업 테이블 만들기

words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
        # 범주 리스트와 해당 인덱스를 전달하여 룩업 테이블을 위한 초기화 객체 만들기
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)
        # 초기화 객체와 버킷을 지정하여 룩업 테이블 만들기
        # 어휘 사전에 없는 범주를 찾으면 룩업 테이블에 추가해주기
        # 최대 추가 개수는 1000개

In [14]:
# 단어 몇 개에 대한 ID 확인해보기

table.lookup(tf.constant([b'This movie was faaaaaantastic'.split()]))

# TF 변환에서 이런 어휘 사전 편리하게 다룰 수 있는 함수 제공
# tft.compute_and_apply_vocabulary로 고유한 모든 단어 찾아 어휘 사전 구축

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   24,    12,    13, 10053]])>

In [15]:
### 최종 훈련 세트 만들기

def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets['train'].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [16]:
### 모델 만들기

embed_size = 128  # 임베딩 행렬의 열이 128개
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           input_shape=[None]),
        # 입력은 [batch size, timesteps] -> 출력은 [batch size, timesteps, embed_size]
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation='sigmoid')
])

# 모델이 패딩 토큰을 무시하도록 학습시키려면 mask_zero=True로 추가해야!
# Sequential 모델이 아니면 직접 마스킹을 계사해서 다음 층에 전달해야!

In [17]:
### 함수형 API에서 직접 마스킹 처리하기

K = keras.backend
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation='sigmoid')(z)
model = keras.Model(inputs=[inputs], outputs=[outputs])

#### 2) 사전훈련된 임베딩 재사용하기
- 텐서플로 허브 프로젝트는 사전훈련된 모델 컴포넌트(모듈)를 모델에 추가하기 쉽게 만들어줌

In [7]:
import tensorflow_hub as hub

In [ ]:
model = keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1',
                   dtype=tf.string, input_shape=[], output_shape=[50]), 
                   # 문장 인코더 (문자열을 입력으로 받아 하나의 벡터로 (50차원))
                   # 모든 단어의 임베딩의 평균을 계산하여 문장 임베딩을 출력
                   # 이 층은 훈련되지 않음 (옵션 추가해서 할 수는 있음)
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# IMDb 데이터를 전처리할 필요가 없고, 바로 모델을 훈련할 수 있음

datasets, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples
batch_size = 32
train_set = datasets['train'].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)

### 3. 신경망 기계 번역을 위한 인코더-디코더 네트워크

In [23]:
# !pip install tensorflow-addons==0.9.1  # 아래코드는 이 버전으로 해야!

In [18]:
import tensorflow_addons as tfa
import numpy as np

/usr/local/lib/python3.6/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.1.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [22]:
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
    # input_dim : 단어 사전의 크기, output_dim : 임베딩 벡터의 크기
    # input_length : 입력 시퀀스의 길이 (다음에 플래튼 레이어가 온다면 반드시 지정해줘야!)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)  
    # 최종 은닉 상태를 디코더로 보내기위해 return_state=True
    # LSTM을 사용하기 때문에 은닉상태 두 개(c_t, h_t)를 반환
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()  # 여러 샘플러 중 하나
    # 이 샘플러는 각 스텝에서 디코더에게 이전 스텝의 출력이 무엇인지 알려줌
    # 예측 시에는 실제로 출력되는 토큰의 임베딩
    # 훈련 시에는 이전 타깃 토큰의 임베딩

    # 실전에서는 ScheduledEmbeddingTrainingSampler와 같이 처음에는 이전 스텝의 
    # 타깃의 임베딩을 사용해 훈련을 시작해서 점차 출력된 토큰의 임베딩으로 바꾸는게 좋음

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer = output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.Model(inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
                    outputs=[Y_proba])
    # decoder_inputs이 들어가는 이유는 훈련시에 이전스텝의 실제값을 넣어주기 때문
    # 만약, 훈련시에 이전스텝의 출력값을 넣어주면 잘못된 예측이 연쇄적으로 일어나 
    # 속도가 느려짐 -> 실제값을 넣어줌!

#### 1) 양방향 RNN
- 문맥을 이해하여 단어의 의미를 파악하기 위해 반대방향으로 층을 하나 더 만듦

In [ ]:
keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True))

#### 2) 빔 검색
- k개의 가능성 있는 문장의 리스트와 조건부확률을 유지하고 단계마다 k개의 문장을 만듦

In [ ]:
beam_width = 10
decoder = tfa.seq2seq.beam_search_decoder.BeamSearchDecoder(
    cell=decoder_cell, beam_width=beam_width, output_layer=output_layer)
decoder_initial_state = tfa.seq2seq.beam_search_decoder.tile_batch(
    encoder_state, multiplier=beam_width)
outputs, _, _ = decoder(
    embedding_decoder, start_tokens=start_tokens, end_token=end_token,
    initial_state=decoder_initial_state
)